In [2]:
import pandas as pd
import os, math
from scipy.stats import t

def convert_to_percentage(value):
    percentage = value * 100
    return f"{percentage:.2f}%"

def DE(model,type0 = 'de'):

    print(model)
    model = model

    Ngr, Ngp, Nlr, Nlp = 0, 0, 0, 0
    sum_equity = 0.0
    count = 0
    
    list_perf = []
    list_de = []
    list_name = []
    list_flag = []
    
    total_buy = 0
    total_sell = 0
    num_buy = 0
    num_sell = 0
    num_both = 0
    num_glr = 0

    for file in os.listdir(model):
        gr, gp, lr, lp = 0, 0, 0, 0

        if 'summary.csv' in file:
            df = pd.read_csv(model+ '/'+file)

            stock_value = list(df['stock_value'])
            buy_price = -1

            equity = list(df['equity'])
            sum_equity += (equity[-1] - 1000000)
            
            list_perf.append(equity[-1] - 1000000)
            name = file[2:file.find("_")]
            
            count += 1
            count_buy = 0
            count_sell = 0
            count_lr = 0
            count_gr = 0

            for i in range(11,len(df)):
             
                if df.at[i-1,'decision'] == "buy":
                    count_buy += 1
                    if df.at[i-1,'hold_num'] != 0: 
                        if df.at[i,'close'] > buy_price: gp += 1
                        else: lp += 1
                   
                    if df.at[i,'hold_num'] != 0:
                        buy_price = (buy_price * df.at[i-1,'hold_num'] + df.at[i,'open'] * (df.at[i,'hold_num'] - df.at[i-1,'hold_num'])) / df.at[i,'hold_num']
                elif df.at[i-1,'decision'] == "sell":
                    if df.at[i-1,'hold_num'] != 0: 
                        count_sell += 1
                        if df.at[i,'close'] >= buy_price: 
                            gr += 1
                            count_gr += 1
                        else: 
                            lr += 1
                            count_lr += 1
                else:
                    if df.at[i-1,'hold_num'] != 0: 
                        if df.at[i,'close'] > buy_price: gp += 1
                        else: lp += 1
                
            tmp_de = 0
            flag = False
            if (gr + gp == 0) and (lr + lp == 0): 
                tmp_de = 0
                flag = True
            elif (gr + gp == 0): 
                tmp_de = 0.5 - (lr)/(lr + lp)
                flag = True
            elif (lr + lp == 0): 
                tmp_de = (gr)/(gr + gp) - 0.5
                flag = True
            else: 
                tmp_de = (gr)/(gr + gp) - (lr)/(lr + lp)
            
            list_de.append(tmp_de)
            list_flag.append(flag==False)
            
            if count_buy!= 0: num_buy += 1
            if count_sell!= 0: num_sell += 1
            total_buy += count_buy
            total_sell += count_sell
            if count_buy!=0 and count_sell!=0: num_both += 1
            if count_gr!=0 and count_lr!=0: num_glr += 1
                
            Ngr += gr
            Ngp += gp
            Nlr += lr
            Nlp += lp
        
    PGR = Ngr / (Ngr + Ngp)
    PLR = Nlr / (Nlr + Nlp)
    DE = PGR - PLR
    # print("Ngr, Ngp, Nlr, Nlp")
    # print(Ngr, Ngp, Nlr, Nlp)
    # print("PGR, PLR")
    # print(PGR, PLR)

    # SE = math.sqrt(PGR*(1-PGR)/(Ngr+Ngp) + PLR*(1-PLR)/(Nlr+Nlp))
    # t = (PGR-PLR)/SE
    re_list_de = [x for x in list_de if x > 0]
    DE = sum(re_list_de)/len(re_list_de)
    # DE = sum(re_list_de)/len(list_de)
    # print('DE', DE, 'return', convert_to_percentage(sum_equity/count/1000000), len(re_list_de), len(list_de), convert_to_percentage(len(re_list_de)/len(list_de)))
    std_dev = math.sqrt(sum((x - DE) ** 2 for x in re_list_de) / len(re_list_de))
    standard_error = std_dev / math.sqrt(len(re_list_de))
    t_statistic = DE / standard_error
    degrees_of_freedom = len(re_list_de) - 1
    p_value = t.sf(abs(t_statistic), df=degrees_of_freedom) * 2
    # print('t_statistic', t_statistic, ' ', 'standard_error', standard_error, ' ', 'p_value', p_value)
    if p_value < 0.001:
        p_value = '$<$0.001'
    if 'csi' in model:
        country = 'China'
    else:
        country = 'USA'
    if 'month' in model:
        k_line = 'monthly'
    if 'week' in model:
        k_line = 'weekly' 
    if 'Observation' in folder:
        print(country,' &', model[-9:], '&', k_line, '&', convert_to_percentage(len(re_list_de)/len(list_de)).replace('%', '\\%'), '&', "{:.3f}".format(round(DE, 3)), '&', "{:.3f}".format(round(t_statistic, 3)), '&', "{:.3f}".format(round(standard_error, 3)), '&', p_value, '&', convert_to_percentage(sum_equity/count/1000000).replace('%', '\\%'), '\\\\')
    if 'Interpretation' in folder:
        print(convert_to_percentage(len(re_list_de)/len(list_de)).replace('%', '\\%'), '&', "{:.3f}".format(round(DE, 3)), '&', "{:.3f}".format(round(t_statistic, 3)), '&', "{:.3f}".format(round(standard_error, 3)), '&', p_value, '&', convert_to_percentage(sum_equity/count/1000000).replace('%', '\\%'), '\\\\')
    if 'Intervention' in folder:
        print('return', convert_to_percentage(sum_equity/count/1000000).replace('%', '\\%'))
        print(' &', "{:.3f}".format(round(DE, 3)), '&', "{:.3f}".format(round(t_statistic, 3)), '&', "{:.3f}".format(round(standard_error, 3)), '&', p_value)
    # print('disposition effect value', DE)
    # print('t-statistic', t_statistic)
    # print('average investment performance', sum_equity/count)
    

    return list_flag, list_de


folder = './Observation_Results'
csi_baseline = './Observation_Results/csi100_gpt-3.5-baseline_monthly_2022-2023'
sp_baseline = './Observation_Results/sp100_gpt-3.5-baseline_monthly_2022-2023'
print("Baseline of China Stock")
__, csi_baseline_de_list = DE(csi_baseline, 'de')
# print(len([x for x in csi_baseline_de_list if x > 0]), len(csi_baseline_de_list), len([x for x in csi_baseline_de_list if x > 0]) / len(csi_baseline_de_list))
print("Baseline of USA Stock")
__, sp_baseline_de_list = DE(sp_baseline, 'de')
# print(len([x for x in sp_baseline_de_list if x > 0]), len(sp_baseline_de_list), len([x for x in sp_baseline_de_list if x > 0]) / len(sp_baseline_de_list))

print('\n The following are the results for different time periods compared to the abover baseline: \n')

for file in os.listdir(folder):
    if 'csi100' in file and csi_baseline != file:
        __, de_list = DE(folder+'/'+file, 'de')


        DE_change = 0
        for i in range(len(csi_baseline_de_list)):
            DE_change += ((de_list[i] - csi_baseline_de_list[i]))
        print('Per.', convert_to_percentage(len([x for x in de_list if x > 0]) / len(de_list)).replace('%', '\\%')) 
        print("**************************************************************\n")
    if 'sp100' in file and sp_baseline != file:
        __, de_list = DE(folder+'/'+file, 'de')
        DE_change = 0
        for i in range(len(sp_baseline_de_list)):
            DE_change += ((de_list[i] - sp_baseline_de_list[i]))
        print('Per.', convert_to_percentage(len([x for x in de_list if x > 0]) / len(de_list)).replace('%', '\\%')) 
        print("**************************************************************\n")

Baseline of China Stock
./Observation_Results/csi100_gpt-3.5-baseline_monthly_2022-2023


China  & 2022-2023 & monthly & 37.66\% & 0.414 & 11.832 & 0.035 & $<$0.001 & 1.64\% \\
Baseline of USA Stock
./Observation_Results/sp100_gpt-3.5-baseline_monthly_2022-2023
USA  & 2022-2023 & monthly & 9.30\% & 0.493 & 6.717 & 0.073 & $<$0.001 & 0.16\% \\

 The following are the results for different time periods compared to the abover baseline: 

./Observation_Results/csi100_gpt-3.5-baseline_monthly_2014-2015
China  & 2014-2015 & monthly & 32.47\% & 0.344 & 10.082 & 0.034 & $<$0.001 & 10.19\% \\
Per. 32.47\%
**************************************************************

./Observation_Results/csi100_gpt-3.5-baseline_monthly_2016-2017
China  & 2016-2017 & monthly & 33.77\% & 0.346 & 9.201 & 0.038 & $<$0.001 & 7.85\% \\
Per. 33.77\%
**************************************************************

./Observation_Results/csi100_gpt-3.5-baseline_monthly_2018-2019
China  & 2018-2019 & monthly & 27.27\% & 0.270 & 9.133 & 0.030 & $<$0.001 & 9.66\% \\
Per. 27.27\%
********************************